In [3]:
import transformers
from transformers import GPT2Tokenizer


In [5]:
jsonlines_file = "data/dev.english.256.head1.jsonlines"
with open(jsonlines_file, "r") as f:
    l = f.readline()
    
tokenizer_dir = "/home/hansonlu/links/data/anagen_models/anagen_anaph_only_b28_lr_default"
tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_dir)

I0314 10:48:17.135134 140524966041408 tokenization_utils.py:312] Model name '/home/hansonlu/links/data/anagen_models/anagen_anaph_only_b28_lr_default' not found in model shortcut name list (gpt2, gpt2-medium, gpt2-large, gpt2-xl, distilgpt2). Assuming '/home/hansonlu/links/data/anagen_models/anagen_anaph_only_b28_lr_default' is a path or url to a directory containing tokenizer files.
I0314 10:48:17.165753 140524966041408 tokenization_utils.py:377] loading file /home/hansonlu/links/data/anagen_models/anagen_anaph_only_b28_lr_default/vocab.json
I0314 10:48:17.166485 140524966041408 tokenization_utils.py:377] loading file /home/hansonlu/links/data/anagen_models/anagen_anaph_only_b28_lr_default/merges.txt
I0314 10:48:17.167036 140524966041408 tokenization_utils.py:377] loading file /home/hansonlu/links/data/anagen_models/anagen_anaph_only_b28_lr_default/added_tokens.json
I0314 10:48:17.167598 140524966041408 tokenization_utils.py:377] loading file /home/hansonlu/links/data/anagen_models/an

In [ ]:
def get_index_remap(tokens_with_tags):
    remap_idx = 0
    res = []
    for t in tokens_with_tags:
        res.append(remap_idx)
        if t != "[CLS]" and t != "[SEP]":
            remap_idx += 1
    return res

def process_jsonline(line):
    example = json.loads(line)
    tokens_with_tags = flatten(example["sentences"])
    token_idx_remap = get_index_remap(tokens_with_tags)
    tokens_without_tags = [t for t in tokens_with_tags if t != "[CLS]" and t != "[SEP]"]
    raw_string_without_tags = " ".join(tokens_without_tags)
    

## Try construct dataset directly from ontonotes data

Try to copy and understand code in minimize.py

In [9]:
import re
import os
import sys
import json
import collections
import conll
from collections import defaultdict as DD

In [10]:
# if __name__ == "main"
data_dir = "/home/hansonlu/links/data/coref_data"
do_lower_case = False
seg_len = 256



In [35]:
class DocumentState(object):
    def __init__(self, key):
        self.doc_key = key
        self.sentence_end = []
        self.token_end = []
        self.tokens = []
        self.subtokens = []
        self.info = []
        self.segments = []
        self.subtoken_map = []
        self.segment_subtoken_map = []
        self.sentence_map = []
        self.pronouns = []
        self.clusters = collections.defaultdict(list)
        self.coref_stacks = collections.defaultdict(list)
        self.speakers = []
        self.segment_info = []

    def finalize(self):
        # finalized: segments, segment_subtoken_map
        # populate speakers from info
        subtoken_idx = 0
        for segment in self.segment_info:
            speakers = []
            for i, tok_info in enumerate(segment):
                if tok_info is None and (i == 0 or i == len(segment) - 1):
                    speakers.append('[SPL]')
                elif tok_info is None:
                    speakers.append(speakers[-1])
                else:
                    speakers.append(tok_info[9])
                    if tok_info[4] == 'PRP':
                        self.pronouns.append(subtoken_idx)
                subtoken_idx += 1
            self.speakers += [speakers]
        # populate sentence map

        # populate clusters
        first_subtoken_index = -1
        for seg_idx, segment in enumerate(self.segment_info):
            speakers = []
            for i, tok_info in enumerate(segment):
                first_subtoken_index += 1
                coref = tok_info[-2] if tok_info is not None else '-'
                if coref != "-":
                    last_subtoken_index = first_subtoken_index + tok_info[-1] - 1
                    for part in coref.split("|"):
                        if part[0] == "(":
                            if part[-1] == ")":
                                cluster_id = int(part[1:-1])
                                self.clusters[cluster_id].append((first_subtoken_index, last_subtoken_index))
                            else:
                                cluster_id = int(part[1:])
                                self.coref_stacks[cluster_id].append(first_subtoken_index)
                        else:
                            cluster_id = int(part[:-1])
                            start = self.coref_stacks[cluster_id].pop()
                            self.clusters[cluster_id].append((start, last_subtoken_index))
        # merge clusters
        merged_clusters = []
        for c1 in self.clusters.values():
            existing = None
            for m in c1:
                for c2 in merged_clusters:
                    if m in c2:
                        existing = c2
                        break
                if existing is not None:
                    break
            if existing is not None:
                print("Merging clusters (shouldn't happen very often.)")
                existing.update(c1)
            else:
                merged_clusters.append(set(c1))
        merged_clusters = [list(c) for c in merged_clusters]
        all_mentions = util.flatten(merged_clusters)
        sentence_map =  get_sentence_map(self.segments, self.sentence_end)
        subtoken_map = util.flatten(self.segment_subtoken_map)
        assert len(all_mentions) == len(set(all_mentions))
        num_words =  len(util.flatten(self.segments))
        assert num_words == len(util.flatten(self.speakers))
        assert num_words == len(subtoken_map), (num_words, len(subtoken_map))
        assert num_words == len(sentence_map), (num_words, len(sentence_map))
        return {
            "doc_key": self.doc_key,
            "sentences": self.segments,
            "speakers": self.speakers,
            "constituents": [],
            "ner": [],
            "clusters": merged_clusters,
            'sentence_map':sentence_map,
            "subtoken_map": subtoken_map,
            'pronouns': self.pronouns
        }

def normalize_word(word, language):
    if language == "arabic":
        word = word[:word.find("#")]
    if word == "/." or word == "/?":
        return word[1:]
    else:
        return word

def get_document(document_lines, tokenizer, language, segment_len):
    document_state = DocumentState(document_lines[0])
    word_idx = -1
    for line in document_lines[1]:
        row = line.split()
        sentence_end = len(row) == 0
        if not sentence_end:
            assert len(row) >= 12
            word_idx += 1
            word = normalize_word(row[3], language)
            subtokens = tokenizer.tokenize(word)
            document_state.tokens.append(word)
            document_state.token_end += ([False] * (len(subtokens) - 1)) + [True]
            for sidx, subtoken in enumerate(subtokens):
                document_state.subtokens.append(subtoken)
                info = None if sidx != 0 else (row + [len(subtokens)])
                document_state.info.append(info)
                document_state.sentence_end.append(False)
                document_state.subtoken_map.append(word_idx)
        else:
            document_state.sentence_end[-1] = True
    # split_into_segments(document_state, segment_len, document_state.token_end)
    # split_into_segments(document_state, segment_len, document_state.sentence_end)
    constraints1 = document_state.sentence_end if language != 'arabic' else document_state.token_end
    split_into_segments(document_state, segment_len, constraints1, document_state.token_end)
    stats["max_sent_len_{}".format(language)] = max(max([len(s) for s in document_state.segments]), stats["max_sent_len_{}".format(language)])
    document = document_state.finalize()
    return document


def minimize_partition(tokenizer, name="dev", language="english", extension="v4_gold_conll", seg_len=256):
    # (original) input_path = ~/links/data/coref_data/dev.english.v4_gold_conll
    # input_path = "{}/{}.{}.{}".format(input_dir, name, language, extension)
    input_path = "/home/hansonlu/anagen/coref/data/dev.english.twodoc.v4_gold_conll"
#     output_path = "{}/{}.{}.{}.jsonlines".format(output_dir, name, language, seg_len)
    count = 0
    print("Minimizing {}".format(input_path))
    documents = []
    with open(input_path, "r") as input_file:
        for line in input_file.readlines():
            begin_document_match = re.match(conll.BEGIN_DOCUMENT_REGEX, line)
            if begin_document_match:
                doc_key = conll.get_doc_key(begin_document_match.group(1), begin_document_match.group(2))
                documents.append((doc_key, []))
            elif line.startswith("#end document"):
                continue
            else:
                documents[-1][1].append(line)
    # documents: List[Tuple[str: doc_key, List[str]]
    
    document_line = get_document(document)
    return
#     with open(output_path, "w") as output_file:
#         for document_lines in documents:
#             if skip(document_lines[0]):
#                 continue
#             document = get_document(document_lines, tokenizer, language, seg_len)
#             output_file.write(json.dumps(document))
#             output_file.write("\n")
#             count += 1
#     print("Wrote {} documents to {}".format(count, output_path))


In [34]:
minimize_partition()

Minimizing /home/hansonlu/anagen/coref/data/dev.english.twodoc.v4_gold_conll
<class 'list'>
540
